In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from pandas import Index
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import request


In [3]:
def signal_process(table,event_var='PM25_Marseille_Longchamp',event_threshold = 40, duration = 6,var_name_list='air pollution'):
    """Find the spike in pm25 and plot a sliding window for google trend """

    index_list=np.where(table[event_var]>event_threshold)[0]
    
    # add first element
    shrink_index_list = [index_list[0]]
    date_set ={table.date[index_list[0]]}
    
    # exclude for the same day
    for cnt in range(1,len(index_list)):
        if table.date[index_list[cnt]] in date_set:
            pass
        else:
            shrink_index_list.append(index_list[cnt].copy())
            date_set.add(table.date[index_list[cnt]])
    
    bind_list = []
    for start_index in shrink_index_list:
        this_sequence = table.loc[range(start_index-2,start_index+duration),
                                  var_name_list]
        start_date = table.datetime[start_index]
        tuples = list(zip([start_date]*(duration+2),range(-2,duration)))

        this_sequence.index = pd.MultiIndex.from_tuples(tuples,names=['event_date_index','hour_index'])
    #         this_sequence = (this_sequence- this_sequence.mean())/this_sequence.std()

        bind_list.append(this_sequence)
    new_table=pd.concat(bind_list)     
    
    return new_table

In [27]:
def fetching_trend(airdata, kw_list,location_code='FR-U',timezone = 0):
    input_table = airdata
    newest_datetime   = max(input_table['datetime'])
    earliest_datetime = min(input_table['datetime'])
    pytrends = TrendReq(hl='En-US', tz=timezone)
    print("fectching trend ")
    ggtrend_list =[]
    for keyword in kw_list:
        
        this_list = [keyword]
        try: 
            topic_code = pytrends.suggestions(keyword)[0]['mid']
            this_list.append(topic_code)
            print("keyword {} is encoded as {}".format(keyword,topic_code))
        except IndexError:
            pass
        print(this_list)
        ggtrend = pytrends.get_historical_interest(this_list, 
                                 year_start=earliest_datetime.year,
                                          month_start=earliest_datetime.month, 
                                          day_start=earliest_datetime.days_in_month, 
                                          hour_start=earliest_datetime.hour, 
                                 year_end=newest_datetime.year,
                                          month_end=newest_datetime.month, 
                                          day_end=newest_datetime.days_in_month, 
                                          hour_end=newest_datetime.hour, 
                                 cat=0, # all categorical
                                          geo=location_code,  # Provence-Alpes-Côte d'Azur
                                          gprop='',  # no specific group # image #hastag etc
                                          sleep=0)
        if len(this_list)==2:
            ggtrend.columns= Index([keyword, keyword+" topic", 'isPartial'], dtype='object')
        
        
        ggtrend_list.append(ggtrend)
    print("trend is fectched")
    return ggtrend_list


In [8]:
# loading table 

In [28]:
parca_table= pd.read_csv("../stage/parca_data.csv")
parca_table=parca_table.loc[(parca_table.station=="Marseille-Longchamp") & (parca_table.measure=="Particules PM2.5")]
parca_table.reset_index(drop=True,inplace=True)
parca_table['datetime']=parca_table['datetime'].apply(lambda string: datetime.strptime(string,'%Y-%m-%d %H:%M:%S'))
parca_table.drop(columns=['station',"measure"],inplace=True)
parca_table.columns=Index(['datetime', 'PM25_Marseille_Longchamp'], dtype='object')
parca_table.head()

,datetime,PM25_Marseille_Longchamp
0,2017-08-08 00:00:00,9.0
1,2017-08-08 01:00:00,7.0
2,2017-08-08 02:00:00,6.0
3,2017-08-08 03:00:00,5.0
4,2017-08-08 04:00:00,4.0


In [29]:
full_date_time = pd.date_range(parca_table[['datetime']].min()[0],parca_table[['datetime']].max()[0],freq='H')
full_time_set = set(map(lambda mytime: mytime.strftime("%Y-%m-%d %H:%M:%S"),full_date_time.to_series())) 
data_time_set = set(map(lambda timestamp: str(timestamp),parca_table['datetime']))
    # pad na values
if len(full_time_set) != len(data_time_set):
    pad_time = full_time_set -data_time_set
    pad_table = pd.DataFrame()
    pad_table['datetime']=[datetime.strptime(mytime,"%Y-%m-%d %H:%M:%S") for mytime in pad_time]
    for column in parca_table.columns:
        if column !="datetime":
            pad_table[column]=None
    parca_table = pd.concat([parca_table,pad_table],ignore_index=True)

In [30]:
# fetching google trend

In [31]:
KW_LIST= ['difficultés respiratoires',
          'difficulté à respirer',
          'asthme',
          'difficulté pour respirer',
          'sensation d’étouffement',
          'dyspnée',
          'spasmophilie',
          'nuage de pollution',
          'saleté de l’air',
          'particules nocives',
          'respiration',
          'maladies respiratoires',
          'pollution de l’air',
          'pollution atmosphérique',
          'pollution',
          'Masque', #mask
          "Purificateur d'air", #air purifier
          
         ]

In [32]:
LOCATION_CODE = 'FR-U'  # ISO 3166 code	FR-U  for Provence-Alpes-Côte d'Azur

In [33]:
ggtrend_list = fetching_trend(parca_table,kw_list=KW_LIST,location_code='FR-U',timezone = -120)


fectching trend 
keyword difficultés respiratoires is encoded as /m/058vp
['difficultés respiratoires', '/m/058vp']
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request fail

ValueError: Length mismatch: Expected axis has 0 elements, new values have 3 elements

In [ ]:
ggtrend=pd.concat(ggtrend_list,axis=1)

In [ ]:
ggtrend.reset_index(inplace=True)

In [ ]:
merged_t=parca_table.merge(ggtrend,left_on='datetime',right_on='date')
kw_str  ="_".join(KW_LIST)
merged_t.to_csv("../stage/parca_{}_{}.csv".format(kw_str,parca_table.columns[1]))

In [ ]:
signal_table=signal_process(merged_t,event_threshold = 35,duration = 12,var_name_list=list(ggtrend.columns)+['PM25_Marseille_Longchamp'])

In [ ]:
signal_table.groupby("hour_index").mean().plot()

In [ ]:
signal_table.to_csv("../output/air_parca_threshold_35_colsum_duration_12H.csv")